Now we have 14 model

For this notebook we will put our picture through ALL the 14 model

and we will then take the three highest (1's) valued models

    by having a 2Darray that hold 14 models x [0 , 1] (softmax) 

function1(){
    return arrayofVal.appended
}

function2(){
    takes poster and turns it into np.array
}

function3(){
    takes arrayOfVals
    returns best three genres with their percentages
}

main(){
input = poster

function2(poster)
arrayyofVals = []
arrayofVals.append (function1(poster, comedy))
function1(poster, action)
function1(poster, horror)
function1(poster, ...)
function1(poster, ...)
function1(poster, ...)
function1(poster, 14...)


function3()


done :)


}


In [23]:
from PIL import Image
import numpy as np
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.models import Sequential
import numpy as np

In [24]:
def PosterToArray(image):
    poster = np.array(image)
    poster = poster.astype('float32') / 255
    return poster

In [25]:
def PredictByModel(ourGenre, posterPath):

    poster = Image.open(posterPath)
    poster = poster.resize((100,100))
    numPyPoster = PosterToArray(poster)
    posterArray = [numPyPoster]
    posterArray = np.array(posterArray) 
    checkpoint_path = f"Weights/{ourGenre}/cp.ckpt"

    model = Sequential()
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(100, 100, 3)))
    model.add(MaxPooling2D((2, 2), padding='same'))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), padding='same'))
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(2, activation="softmax"))
    model.compile(optimizer='adam', loss='CategoricalCrossentropy', metrics=["accuracy"])

    model.load_weights(checkpoint_path).expect_partial()
    
    predicted = model.predict(posterArray)
    return (predicted[0], ourGenre)
    
    

In [26]:
def PredictAllModes(posterPath, allPredictions):
    Genres = ['Action', 'Adventure', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Horror', 'Music', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller']
    for i in range(len(Genres)):
        allPredictions.append(PredictByModel(Genres[i], posterPath, allPredictions))
    return allPredictions

In [27]:
def ThreeGenres(allPredictions):
    tempSort = []
    for i in range(len(allPredictions)):
        temp = [allPredictions[i][0][1] , allPredictions[i][1]]
        tempSort.append(temp)
    tempSort.sort()
    tempSort.reverse()
    print(f"Genre1: {tempSort[0][1]} ({np.round((tempSort[0][0] * 100),2)}%)    Genre2: {tempSort[1][1]} ({np.round(tempSort[1][0] * 100,2)}%)    Genre3: {tempSort[2][1]}({np.round(tempSort[2][0] * 100,2)}%)")
    
    

In [29]:
Genres = ['Action', 'Adventure', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Horror', 'Music', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller']
posterPath = f"test1.jpg"
allPredictions = []
for i in range(len(Genres)):
    allPredictions.append(PredictByModel(Genres[i], posterPath))
allPredictions

[(array([0.22555223, 0.7744478 ], dtype=float32), 'Action'),
 (array([0.49388507, 0.50611496], dtype=float32), 'Adventure'),
 (array([0.9744942 , 0.02550577], dtype=float32), 'Biography'),
 (array([0.47858185, 0.52141815], dtype=float32), 'Comedy'),
 (array([0.6587986 , 0.34120142], dtype=float32), 'Crime'),
 (array([0.76437545, 0.23562455], dtype=float32), 'Documentary'),
 (array([0.14763921, 0.85236084], dtype=float32), 'Drama'),
 (array([0.5923431 , 0.40765688], dtype=float32), 'Family'),
 (array([0.27431646, 0.72568357], dtype=float32), 'Fantasy'),
 (array([0.80459017, 0.19540986], dtype=float32), 'Horror'),
 (array([0.3660232, 0.6339768], dtype=float32), 'Music'),
 (array([0.6455122 , 0.35448784], dtype=float32), 'Mystery'),
 (array([0.6166978, 0.3833022], dtype=float32), 'Romance'),
 (array([0.13851996, 0.86148   ], dtype=float32), 'Sci-Fi'),
 (array([0.274306  , 0.72569406], dtype=float32), 'Thriller')]

In [30]:
ThreeGenres(allPredictions)

Genre1: Sci-Fi (86.15%)    Genre2: Drama (85.24%)    Genre3: Action(77.44%)
